In [3]:
import requests
import pandas as pd
import json
import time
import pyodbc 
import urllib
import os
import bs4
import datetime

In [4]:
## Faz o download do conteúdo da página
url = "https://store.steampowered.com/hwsurvey/videocard/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = bs4.BeautifulSoup(content, "html.parser")


In [5]:
##extrai dados das divs
data_list1 = []
for i in range(0,2,1) :
# Encontra todas as divs com a classe "substats_row row_0"
    divs = soup.find_all("div", class_=f"substats_row row_0")

    # Extrai os dados de cada div
    for div in divs:
        data = div.get_text(separator='|', strip=True)
        data_list1.append(data)

data_list2 = []
for i in range(0,2,1) :
# Encontra todas as divs com a classe "substats_row row_0"
    divs = soup.find_all("div", class_=f"substats_row row_1")

    # Extrai os dados de cada div
    for div in divs:
        data = div.get_text(separator='|', strip=True)
        data_list2.append(data)

##combina as duas listas em uma unica lista
dados_combinados = [item for pair in zip(data_list1, data_list2) for item in pair]
df = pd.DataFrame(dados_combinados,columns=['GPU_DATA']) 

df
        

,GPU_DATA
0,DirectX 12 GPUs|90.82%|90.40%|90.18%|88.65%|88...
1,DirectX 11 GPUs|1.38%|1.39%|1.31%|1.44%|1.26%|...
2,DirectX 10 GPUs|0.82%|0.80%|0.79%|0.86%|0.74%|...
3,DirectX 9 Shader Model 2b and 3.0 GPUs|0.02%|0...
4,DirectX 9 Shader Model 2.0 GPUs|0.00%|0.00%|0....
...,...
531,VIA/S3G UniChrome Pro IGP|15.63%|3.45%|8.33%|1...
532,NVIDIA GeForce 6150 LE|0.51%|-|-|-|0.63%|+0.63%
533,ATI Mobility Radeon X600|-|-|-|-|4.00%|+4.00%
534,NVIDIA GeForce Go 6150|-|-|-|-|0.63%|+0.63%


In [ ]:
##armazena dados bronze stage
os.makedirs(f'assets/gpu_data/bronze_raw/date_ref_{datetime.date.today()}')
df.to_json(f'assets/gpu_data/bronze_raw/date_ref_{datetime.date.today()}/dados_ranking_gpu_{datetime.date.today()}.json')


##SILVER STAGE

In [ ]:
bronze_content = pd.read_json(f'assets/gpu_data/bronze_raw/date_ref_{datetime.date.today()}/dados_ranking_gpu_{datetime.date.today()}.json', orient='values').sort_index()
df = bronze_content['GPU_DATA'].str.split('|',expand=True)

# #renomeia colunas
df.columns = ['Placa','MES_1','MES_2','MES_3','MES_4','MES_ATUAL','RESULTADO']

##remove as 6 primeiras linhas
df = df.drop(index=range(6)).reset_index(drop=True)

# # #identifica quais linhas quero remover
remover = df.query('Placa.str.contains("Other")').index


# ##MARCA A PARTIR DE ONDE QUERO EXCLUIR
ultima_linha = remover[0]
df = df[:ultima_linha+1]


##CRIA DICIONARIO COM AS SUBSTITUICOES QUE QUERO FAZER NAS COLUNAS DO DATAFRAME
substituicoes = {'%':'','-':'0'}
df.iloc[0:,1:].replace(substituicoes,regex=True,inplace=True)


##CRIA NOVA COLUNA DE RESULTADO FLOAT
df.drop(columns='RESULTADO')
df['RESULTADO'] = (df['MES_ATUAL'].astype(float)  - df['MES_4'].astype(float)).round(2)

##SEPARA POR MARCA
marca = df['Placa'].str.split(' ',n=1,expand=True)[0]
placa = df['Placa'].str.split(' ',n=1,expand=True)[1]
df['Marca'] = marca
df['Placa'] = placa

##corrigi classificação intel
df['Marca'].replace('Intel(R)', 'INTEL',inplace=True)
df['Marca'].replace('Other', 'OUTRAS', inplace=True)

##reclassifica registro sem nenhuma placa
df['Placa'].fillna('OUTRAS', inplace=True)


df

In [ ]:
##armazena dados silver stage
os.makedirs(f'assets/gpu_data/silver_stage/date_ref_{datetime.date.today()}')
df.to_csv(f'assets/gpu_data/silver_stage/date_ref_{datetime.date.today()}/dados_ranking_gpu_tratado_{datetime.date.today()}.csv',index=None)


##GOLD STAGE 

In [14]:
silver_content = pd.read_csv(f'assets/gpu_data/silver_stage/date_ref_{datetime.date.today()}/dados_ranking_gpu_tratado_{datetime.date.today()}.csv')
silver_content


,Placa,MES_1,MES_2,MES_3,MES_4,MES_ATUAL,RESULTADO,Marca
0,GeForce GTX 1650,6.19,6.07,5.67,5.30,5.11,-0.19,NVIDIA
1,GeForce RTX 3060,4.66,4.90,4.74,4.88,4.87,-0.01,NVIDIA
2,GeForce GTX 1060,4.99,4.94,4.58,4.60,4.31,-0.29,NVIDIA
3,GeForce RTX 3060 Laptop GPU,4.51,4.54,4.28,3.76,4.29,0.53,NVIDIA
4,GeForce RTX 2060,4.45,4.53,4.15,4.10,3.94,-0.16,NVIDIA
...,...,...,...,...,...,...,...,...
94,UHD Graphics 600,0.16,0.17,0.16,0.18,0.17,-0.01,INTEL
95,Radeon R7 Graphics,0.17,0.16,0.16,0.17,0.16,-0.01,AMD
96,GeForce GTX 750,0.19,0.17,0.18,0.17,0.16,-0.01,NVIDIA
97,HD Graphics 530,0.00,0.00,0.15,0.17,0.16,-0.01,Intel


In [ ]:

##cria conexão com o banco

DRIVER = 'SQL SERVER'
SERVER = 'DESKTOP-2VSV2T8'
DB_NAME = 'Projeto_Steam'

conn = pyodbc.connect(f"""
        DRIVER={{{DRIVER}}};
        SERVER={SERVER};
        DATABASE={DB_NAME};
        Trust_Connection=yes;
                        """)

##cria cursor para execução dos comandos em sql
cursor = conn.cursor()

##comita para salvar as alterações
cursor.commit()

In [ ]:
##cria tabela para as informações das placas

cursor.execute('drop table if exists tb_placas_de_video_resultado  create table Tb_Placas_De_Video_Resultado (Marca varchar(100), placa varchar(200), ABRIL NUMERIC(20,2),MAIO NUMERIC(20,2),JUNHO NUMERIC(20,2),JULHO NUMERIC(20,2),AGOSTO NUMERIC(20,2),RESULTADO NUMERIC(20,2))')
cursor.commit()

##Insere dados nas tabelas

for index,row in df.iterrows():
    cursor.execute(f'insert into tb_placas_de_video_resultado(Marca,placa,ABRIL,MAIO,JUNHO,JULHO,AGOSTO,RESULTADO) values(?,?,?,?,?,?,?,?)',row[7],row[0],row[1],row[2],row[3],row[4],row[5],row[6])

cursor.commit()

##ATUALIZACAO MENSAL

In [ ]:
##ADICIONA TABELA DO MES
cursor.execute('alter table tb_placas_de_video_resultado add  AGOSTO NUMERIC(20,2)')
cursor.commit()

##CRIA TABELA TEMPORARIA PARA ARMAZENAR O RESULTADO DO  MES
cursor.execute('drop table if exists tb_placas_de_video_resultado_temp create table Tb_Placas_De_Video_Resultado_temp (Marca varchar(200),placa varchar(200), MES_ATUAL NUMERIC(20,2),RESULTADO_MES NUMERIC(20,2))')
cursor.commit()


In [ ]:
##insere na tabela temporaria os dados do mês atual
for index,row in df.iterrows():
    cursor.execute(f'insert into tb_placas_de_video_resultado_temp(Marca,placa,MES_ATUAL,RESULTADO_MES) values(?,?,?,?)',row.Marca,row.Placa,row.MES_ATUAL,row.RESULTADO)

cursor.commit()

In [ ]:
##atualiza tabela principal

cursor.execute("""
               update tb_placas_de_video_resultado_temp
               set placa = 'Outras'
               where placa = 'OUTRAS'""")

#insere placas que não existiam em outros meses, lembrar de acrescentar os null
cursor.execute("""
                Insert into tb_placas_de_video_resultado 
                select Marca, 
                placa
                from tb_placas_de_video_resultado_temp
                where placa not in (select placa from tb_placas_de_video_resultado )
                """)

cursor.execute("""update tb_placas_de_video_resultado 
                  set AGOSTO = MES_ATUAL, RESULTADO = RESULTADO_MES
                  FROM tb_placas_de_video_resultado a
                  inner join tb_placas_de_video_resultado_temp b  on a.placa = b.placa   """)
cursor.commit()